# Ray Concepts - Data Parallelism (Part 1)

Now let's explore Ray's core concepts and understand how they work. As much as possible, Ray tries to leverage familiar Python idioms, extending them as necessary.

First, we need to import `ray` and we'll also import the `time` API. (If you get an error in the next cell, make sure you set up the tutorial as described in the project [README](../README.md).

> **Tip:** The [Ray Package Reference](https://ray.readthedocs.io/en/latest/package-ref.html) in the [Ray Docs](https://ray.readthedocs.io/en/latest/) is useful for exploring the API features we'll learn.

In [3]:
import ray, time

Now consider the following Python function, where we simulate doing something that's slow to complete, using the `sleep` method. A real world example might do a complex calculation (like a training step for machine learning) or call an external web service where a response could take many milliseconds. We'll use more interesting examples later.

In [4]:
def expensive(n):
    start = time.time()       # Let's time how long this takes.
    time.sleep(n)             # Sleep for n seconds
    return (n, time.time() - start)   # Return n and the duration in seconds

Let's define a helper function to show results:

In [5]:
def p(n, duration):
    print('{:2d}: {:6.3f} seconds'.format(n, duration))

In [6]:
(n, duration) = expensive(2)
p(n, duration)

 2:  2.004 seconds


You should see the output `2: 2.00X seconds`, where `X` is small. As we would expect, it took about two seconds to execute.

Now suppose we need to fire off five of these at once:

In [7]:
start_all = time.time()
for n in range(5):
    n2, duration = expensive(n)
    p(n, duration)
print("Total time:")
p(10, time.time() - start_all)

 0:  0.000 seconds
 1:  1.002 seconds
 2:  2.005 seconds
 3:  3.004 seconds
 4:  4.004 seconds
Total time:
10: 10.015 seconds


It takes about 10 seconds to run, because we do this process _synchronously_, but we don't need to do this. Each call to `expensive()` is independent of the others, so ideally we should run them in _parallel_, i.e., _asynchronously_, so all of them finish more quickly.

Ray makes this easy. Let's define a new function and annotate it with `@ray.remote`. In Ray terminology, the annotation converts the function to a _task_, because we'll now be able to let Ray schedule this "task" (i.e., unit of work) on any CPU core in our laptop or in our cluster when we use one.

In [8]:
@ray.remote
def expensive_task(n):
    return expensive(n)

Note that we could simply call `expensive()`, we don't have to redefine it.

Now when we invoke `expensive_task`, we have to use `expensive_task.remote(n)` instead of `expensive_task(n)`, like before. Python is malleable; the Ray team could have instrumented `expensive_task` so that we can call it like a normal function, but the explicit `.remote` is reminder to the reader what code is using Ray vs. normal Python code.

Okay, let's try the same loop as before. But first, we have to initialize Ray:

In [9]:
ray.init()

2020-04-05 14:03:39,192	INFO resource_spec.py:212 -- Starting Ray with 4.15 GiB memory available for workers and up to 2.1 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-05 14:03:39,554	INFO services.py:1148 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:12703',
 'object_store_address': '/tmp/ray/session_2020-04-05_14-03-39_183605_59740/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-05_14-03-39_183605_59740/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-04-05_14-03-39_183605_59740'}

> **Troubleshooting**
>
> 1. If you get an error like `... INFO services.py:... -- Failed to connect to the redis server, retrying.`, it probably means you are running a VPN on your machine. [At this time](https://github.com/ray-project/ray/issues/6573), you can't use `ray.init()` with a VPN running. You'll have to stop your VPN for now.
> 
> 2. If `ray.init()` worked (for example, you see a message like _View the Ray dashboard at localhost:8265_) and you're using a Mac, you may get several annoying dialogs asking you if you want to allow incoming connections for Python and/or Redis. Click "Accept" for each one and they shouldn't appear again during this lesson. MacOS is trying to verify if these executables have been properly signed. Ray uses Redis. If you installed Python using Anaconda or other mechanism, then it probably isn't properly signed from the point of view of MacOS. To permanently fix this problem, [see this StackExchange post](https://apple.stackexchange.com/questions/3271/how-to-get-rid-of-firewall-accept-incoming-connections-dialog).

If `ray.init()` worked successfully, you'll see a JSON block with information such as the `node_ip_address` and `webui_rul`. 

A separate message tells you that URL is for the Ray dashboard. Open it now in a separate browser tab. It should look something like this:
![Ray Dashboard screenshot](../images/Ray-Dashboard.png)

> **Tip:** You can ask Ray for this URL later if needed. Use `ray.get_webui_url()`.
> 
> **Note:** There are many options you can pass to `ray.init()`. See [the docs](https://ray.readthedocs.io/en/latest/configure.html) for details, some of which we'll explore in later modules.

My laptop has four cores, each of which has a hardware _thread_, for a total of eight. Ray started a `ray` worker process for each hardware thread. These workers are used to run tasks. Click around the dashboard, especially when we run tasks like we're about to do. We'll explore the dashboard more later on. Many laptops have eight cores, so if you may see 16 ray processes.

Now let's run our new Ray task!

In [10]:
expensive_task.remote(2)

ObjectID(45b95b1c8bd3a9c4ffffffff010000c801000000)

What's this `ObjectID` thing? Recall that `expensive()` returned `(n, seconds)`. Now, when we invoke a task, it will be executed _asynchronously_, so instead of the tuple we will eventually want, we get a reference to a Python [Future](https://docs.python.org/3/library/asyncio-future.html), which we'll use to retrieve the tuple when the task has completed. One way to do this is to use `ray.get()`. So, let's modify our previous loop to use the task and retrieve the values using the futures.

In [11]:
start_all = time.time()
for n in range(5):
    id = expensive_task.remote(n)  # Call the remote task
    n2, duration = ray.get(id)     # Retrieve the value using the future
    p(n, duration)
print("Total time:")
p(10, time.time() - start_all)

 0:  0.000 seconds
 1:  1.001 seconds
 2:  2.004 seconds
 3:  3.004 seconds
 4:  4.005 seconds
Total time:
10: 11.069 seconds


I said that Ray would make everything go faster, but the performance is the same. The reason is because we used `ray.get()` incorrectly. This is a _blocking call_; we're telling Ray, "I need the value and I'm going to wait until the task is done and you can return it to me." Making this blocking call in the loop defeats the goal of leveraging asynchrony.

Instead, we need to "fire off" all the asynchronous calls, building up a list of futures, then wait for all of them at once. We'll do that as follows, where for this purposes we'll introduce a list comprehension to call the tasks:

In [12]:
start_all = time.time()
ids = []
for n in range(5):
    id = expensive_task.remote(n)
    ids.append(id)
    p(n, time.time() - start_all)

for n2, duration in ray.get(ids):    # Retrieve all the values for a list of futures
    p(n2, duration)
print("Total time:")
p(10, time.time() - start_all)

 0:  0.001 seconds
 1:  0.001 seconds
 2:  0.002 seconds
 3:  0.003 seconds
 4:  0.003 seconds
 0:  0.000 seconds
 1:  1.002 seconds
 2:  2.005 seconds
 3:  3.005 seconds
 4:  4.005 seconds
Total time:
10:  4.010 seconds


Notice what happened. In the first loop, when we called `expensive_task.remote(n)`, each call returned immediately, so the "durations" were tiny. Then you probably noticed that nothing happend for about four seconds, then suddenly everything was printed, for a total elapsed time of about four seconds. 

Why four? When we pass a list of futures to `ray.get()`, it blocks until the results are available for _all_ of them. Our longest task was four seconds, so once that one finished, the others were already done and all could be returned immediately.

Run the next cell, which is basically the same calculation, but it uses a more idiomatic list comprehension for the `expensive_task` invocations and doesn't log the times for those calls, as we now know these times are trivial.

**However**, as soon as the call starts, switch to the Ray Dashboard browser tab and watch what happens (or use separate windows for these two tabs). You notice instances of `expensive_task` being executed by the different `ray` processes. You'll note that Try using a larger number than `5` so it's easier

In [13]:
start_all = time.time()
ids = [expensive_task.remote(n) for n in range(5)]
for n2, duration in ray.get(ids):    # Retrieve all the values for a list of futures
    p(n2, duration)
print("Total time:")
p(10, time.time() - start_all)

 0:  0.000 seconds
 1:  1.004 seconds
 2:  2.005 seconds
 3:  3.003 seconds
 4:  4.004 seconds
Total time:
10:  4.008 seconds


## Exercise 1

Let's make sure you understand how to use Ray's task parallelism. In the following two cells, we define a new Python function and then use it several times to perform work. Modify both cells to use Ray. The third cell uses `assert` statements to check your work.

> **Tip:** The solution is in the `solutions` folder.

In [14]:
def slow_square(n):
    time.sleep(n)
    return n*n

In [19]:
start = time.time()
squares = [slow_square(n) for n in range(4)]
duration = time.time() - start

In [23]:
assert squares == [0, 1, 4, 9]
# should fail until the code modifications are made:
assert duration < 4.1, f'duration = {duration}' 

AssertionError: duration = 6.012396812438965

## A Closer Look at Scheduling

> **Note:** The rest of this lesson (notebook) isn't essential for now, so you can skip it if you find it challenging to follow.

To better see what's happening with the dashboard, run the following cells to determine the number of CPU hardware threads on your laptop, each of which is running a `ray` process. We've expanded this code over several cells so you can see what each step returns, but you could write it all at once, `num_cpus = ray.nodes()[0]['Resources']['CPU']`.


In [62]:
import json
nodes = ray.nodes()  # Get a JSON object with metadata about all the nodes in your "cluster".
nodes                # On your laptop, a list with one node.

[{'NodeID': 'c6d96a98066c604a345cc03ccb35bebe1e412a2f',
  'Alive': True,
  'NodeManagerAddress': '192.168.1.149',
  'NodeManagerHostname': 'DWAnyscaleMBP.local',
  'NodeManagerPort': 65533,
  'ObjectManagerPort': 49185,
  'ObjectStoreSocketName': '/tmp/ray/session_2020-04-05_09-12-13_691124_26550/sockets/plasma_store',
  'RayletSocketName': '/tmp/ray/session_2020-04-05_09-12-13_691124_26550/sockets/raylet',
  'Resources': {'object_store_memory': 31.0,
   'CPU': 8.0,
   'node:192.168.1.149': 1.0,
   'memory': 91.0},
  'alive': True}]

In [63]:
node = nodes[0]    # Get the single node
node

{'NodeID': 'c6d96a98066c604a345cc03ccb35bebe1e412a2f',
 'Alive': True,
 'NodeManagerAddress': '192.168.1.149',
 'NodeManagerHostname': 'DWAnyscaleMBP.local',
 'NodeManagerPort': 65533,
 'ObjectManagerPort': 49185,
 'ObjectStoreSocketName': '/tmp/ray/session_2020-04-05_09-12-13_691124_26550/sockets/plasma_store',
 'RayletSocketName': '/tmp/ray/session_2020-04-05_09-12-13_691124_26550/sockets/raylet',
 'Resources': {'object_store_memory': 31.0,
  'CPU': 8.0,
  'node:192.168.1.149': 1.0,
  'memory': 91.0},
 'alive': True}

In [64]:
resources = node['Resources']   # Get the resources for the node
resources

{'object_store_memory': 31.0,
 'CPU': 8.0,
 'node:192.168.1.149': 1.0,
 'memory': 91.0}

In [65]:
num_cpus = resources['CPU']  # Get the number of CPU hardware threads
num_cpus

8.0

The final number will be `8.0`, `16.0`, etc. The next cell is one of our previous examples of calling `expensive_task`, but now the loop counter is `2*int(num_cpus)` instead of `5`. This will mean that half of the tasks will have to wait for an open slot. Now run the following cell and watch the Ray dashboard. (You'll know the cell is finished when all the `ray` workers return to `IDLE`.)

What's the total time now? How about the individual times?

In [61]:
start_all = time.time()
ids = []
for n in range(2*int(num_cpus)):     # What's changed!
    id = expensive_task.remote(n)
    ids.append(id)
    p(n, time.time() - start_all)

for n2, duration in ray.get(ids):    # Retrieve all the values for a list of futures
    p(n2, duration)
print("Total time:")
p(10, time.time() - start_all)

 0:  0.001 seconds
 1:  0.002 seconds
 2:  0.003 seconds
 3:  0.005 seconds
 4:  0.008 seconds
 5:  0.008 seconds
 6:  0.010 seconds
 7:  0.011 seconds
 8:  0.012 seconds
 9:  0.013 seconds
10:  0.014 seconds
11:  0.014 seconds
12:  0.015 seconds
13:  0.015 seconds
14:  0.015 seconds
15:  0.018 seconds
 0:  0.000 seconds
 1:  1.004 seconds
 2:  2.003 seconds
 3:  3.002 seconds
 4:  4.005 seconds
 5:  5.002 seconds
 6:  6.003 seconds
 7:  7.004 seconds
 8:  8.003 seconds
 9:  9.003 seconds
10: 10.004 seconds
11: 11.004 seconds
12: 12.004 seconds
13: 13.001 seconds
14: 14.002 seconds
15: 15.002 seconds
Total time:
10: 22.031 seconds


On my 8-worker machine, 16 tasks were run.

Look at the first set of times, for the submissions. They are still fast and nonblocking, but on my machine they took about ~0.02 seconds to complete, so some competition for CPU time occurred.

As before, each asynchronous task still takes roughly `n` seconds to finish (for `n` equals 0 through 15). This makes sense, because each `expensive_task` does essentially nothing but sleep, and since there's only one task per worker, there should be no apreciable difference for the individual times, as before. 

However, the whole process took about 22 seconds, not 16, as we might have expected from our previous experience (i.e., the time for the longest task). This reflects the fact that half the tasks had to wait for an available worker.

In fact, we can explain the 22 seconds exactly. Here is how my 16 tasks, with durations 0 to 15 seconds, were allocated to the 8 workers. Keep in mind that the scheduling happened in order for increasing `n`.

The first 8 tasks, of duration 0 to 7 seconds, where scheduled immediately in the 8 available workers. The 0-second task finished immediately, so the next waiting task, the 8-second task was scheduled on that worker. It finished in 8 seconds, so the _total_ time for the 0-second and 8-second tasks was about 8 seconds. Similarly, after the 1-second task finished, the 9-second task was scheduled. Total time: 10 seconds. Using induction ;), the last worker started with the 7-second task followed by the 15-second task for a total of 22 seconds!

Here's a table showing this in detail. where `n1` and `n2` refers to the first and second tasks, with durations `n1` seconds and `n2` seconds, for a total of `n1+n2` seconds. For consistency, the `ray` workers are numbered from zero:

| Worker | n1 | n2 | Total Time |
| -----: | -: | -: | ---------: |
| 0 | 0 |  8 |  8 |
| 1 | 1 |  9 | 10 |
| 2 | 2 | 10 | 12 |
| 3 | 3 | 11 | 14 |
| 4 | 4 | 12 | 16 |
| 5 | 5 | 13 | 18 |
| 6 | 6 | 14 | 20 |
| 7 | 7 | 15 | 22 |



Of course a real-world scheduling scenario would be more complicated, but hopefully you have a better sense of how Ray distributes work, whether you're working on a single laptop or a large cluster!

In [66]:
import numpy as np

In [74]:
@ray.remote
def make_array(n):
    return np.random.standard_normal(n)

In [79]:
@ray.remote
def add_array(a1, a2):
    return np.add(a1, a2)

In [80]:
start = time.time()
id1 = make_array.remote(50)
id2 = make_array.remote(50)
id3 = add_array.remote(id1, id2)
p(0, time.time() - start)

 0:  0.004 seconds


In [81]:
ray.get(id3)

array([ 0.89481924,  0.31457393, -1.69722375,  1.15585332,  0.0674097 ,
       -3.33268178,  1.24818356,  0.4843486 ,  0.2956283 , -1.42737235,
        0.51966492,  2.10530407, -0.14250201,  1.15019645, -1.98355661,
        1.0644615 , -0.28244692, -0.50563297,  0.25035852, -0.31200625,
       -0.07985132,  1.24236184, -3.2291332 , -0.44351917, -1.90934927,
       -0.98713613, -0.31845726,  1.30390042,  2.64642879,  2.38873804,
        1.65621251,  0.40365107, -0.15567494,  0.55067068, -0.53992095,
       -0.47219108, -0.43920156,  1.72588612,  2.04282036,  0.00481064,
       -1.71421628,  0.63351087,  0.4089416 ,  0.7104193 ,  0.40053558,
        1.55510326,  0.34936558, -2.66941684,  0.8164263 , -0.22020159])

In [82]:
p(1, time.time() - start)

 1:  1.874 seconds


In [72]:
make_array2(5)

TypeError: 'module' object is not callable

In [73]:
np.random.standard_normal(10)

array([-1.47778577, -1.08826959,  0.99370832,  0.29782383, -0.33399421,
       -0.59041933, -0.00511437, -1.64825151, -0.52843183,  1.02245258])